# Майнор ВШЭ 

## Прикладные задачи анализа данных 2020

## Семинар 10: Детектирования аномалий

- [Часть 0. Введение](#Часть-0.-Введение)
- [Часть 1. Простые методы](#Часть-1.-Простые-методы)
    - [1.1. Box plot (ящик с усами)](#1.1.-Box-plot-(ящик-с-усами))
    - [1.2. Z-score](#1.2.-Z-score)
- [Часть 2. Elliptic Envelope](#Часть-2.-Elliptic-Envelope)
- [Часть 3. Одноклассовый SWM](#Часть-3.-Одноклассовый-SWM)
- [Часть 4. Изолирующий лес (Isolation Forest)](#Часть-4.-Изолирующий-лес-(Isolation-Forest))
- [Часть 5. Раздеяющие поверхности для разных алгоритмов](#Часть-5.-Раздеяющие-поверхности-для-разных-алгоритмов)
    - [5.1. OneClassSVM](#5.1.-OneClassSVM)
    - [5.2. IsolationForest](#5.2.-IsolationForest)
    - [5.3. EllipticEnvelope](#5.3.-EllipticEnvelope)
- [Часть 6. Библиотека PyOD](#Часть-6.-Библиотека-PyOD)
    - [6.1. kNN](#6.1.-kNN)

In [ ]:
import numpy as np
from numpy import genfromtxt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

## Часть 0. Введение

**Задача поиска аномалий** (Anomaly Detection) - один из вариантов обучения без учителя (Unsupervised Learning):
обычно примеров аномалий или нет, или их достаточно мало.

Можно выделить два направления, занимающихся поиском аномалий:
1. Детектирование выбросов ([Outlier Detection](https://en.wikipedia.org/wiki/Anomaly_detection))
2. Детектирование «новизны» ([Novelty Detection](https://en.wikipedia.org/wiki/Novelty_detection))

В статистике **выбросом** называю результат измерения, выделяющийся из общей выборки.

**Выбросы** являются следствием:

- ошибок в данных (неточности измерения, округления, неверной записи и т.п.)
- наличия шумовых объектов (неверно классифицированных объектов)
- присутствия объектов «других» выборок (например, показаниями сломавшегося датчика).

**«Новый объект»**, как и **выброс**,  — это объект, который отличается по своим свойствам от объектов (обучающей) выборки.  Но  в отличие от выброса, его в самой выборке пока нет (он появится через некоторое время, и задача как раз и заключается в том, чтобы обнаружить его при появлении). **Новизна**, как правило, появляется в результате принципиально нового поведения объекта.

<img src="https://alexanderdyakonov.files.wordpress.com/2017/04/fig_softoutlier.png" width="500">

На рисинке видно, что шум (noise) — это выброс «в слабом смысле» (он может немного размывать границы класса/кластера). Нас же интересуют, прежде всего, выбросы «в сильном смысле», которые искажают эти границы.

Практические приложения:
- Обнаружение подозрительных банковских операций (Credit-card Fraud)
- Обнаружение вторжений (Intrusion Detection)
- Обнаружение нестандартных игроков на бирже (инсайдеров)
- Обнаружение неполадок в механизмах по показаниям датчиков
- Медицинская диагностика (Medical Diagnosis)
- Сейсмология

## Часть 1. Простые методы

### 1.1. Box plot (ящик с усами)

Будем работать с набором данных [Boston house prices dataset](https://scikit-learn.org/stable/datasets/index.html#boston-dataset).

In [ ]:
from sklearn.datasets import load_boston

In [ ]:
# Загружаем данные
boston = load_boston()

X = boston.data
y = boston.target

# Посмотрим на данные
columns = boston.feature_names
boston_df = pd.DataFrame(boston.data)
boston_df.columns = columns
boston_df.head()

#### Построение Box plot

<img src="https://upload.wikimedia.org/wikipedia/commons/1/1a/Boxplot_vs_PDF.svg?uselang=ru" width="500">

Построим **box plot** для признака `DIS`.

In [ ]:
sns.boxplot(x=boston_df['DIS']);

Как видно из графика, присутствует некоторое количество выбросов. Такой способ удобен для визуализации, но не для нахождения самих выбросов.

Исходя из методики построения ящика с усами, выбросами считаются точки, чьё значение признака $x$:
1. $x > Q_3 + 1.5*IQR$
2. $x < Q_1 - 1.5*IQR$

где **интерквартильный размах** $IQR = Q_3 - Q_1$

**Задание 1**

1. Найдите индексы выбросов для признака `DIS`
2. Исключите выбросы
3. Постройте box plot для `DIS`

In [ ]:
#------------------------------------------------
# Ваш код здесь
# outlier_idx = 
#------------------------------------------------

### 1.2. Z-score

**Стандартизованная оценка** (z-оценка, standard score, z-score) - это мера относительного разброса наблюдаемого или измеренного значения, которая показывает сколько стандартных отклонений составляет его разброс относительного среднего значения.

**Интуиция.**
Вычисляя Z-score, мы масштабируем и центрируем данные и смотрим на точки, которые находятся далеко от 0. 

$$\Large z = \frac{x - \bar{x}}{std}$$

где
- $\bar{x}$ - выборочное среднее
- $std$ - выборочное стандартное отклонение

Точки, которые достаточно далеко от 0, считаются выбросами. В большинстве случаев используется порог 3 или -3, т.е. если **Z-score** больше 3 или меньше -3, то точка считается выбросом.

**Замечание:** *`правило трёх сигм` гласит, что приблизительно с вероятностью 0,9973 значение нормально распределённой случайной величины лежит в интервале $\pm 3\sigma$).*


<img src="https://upload.wikimedia.org/wikipedia/commons/3/37/Standard_deviation_diagram_%28decimal_comma%29.svg?uselang=ru" width="500">

**Задание 2**

1. Найдите индексы выбросов для признака `DIS` используя **z-score**
2. Исключите выбросы
3. Постройте box plot для `DIS`

In [ ]:
#------------------------------------------------
# Ваш код здесь
# outlier_idx = 
#------------------------------------------------

## Часть 2. Elliptic Envelope 

Будем считать, что распределение данных подчиняется многомерному нормальному распределению. Будем оценивать параметры этого распределения $\mu$ и $\Sigma$ по данным, где

- $\mu$ - вектор средних значений
- $\Sigma$ - ковариационная матрица.

Плотность вероятности:

$$\Large f_X(x_1, x_2, ...,x_k) = \frac{1}{\sqrt{(2\pi)^k |\Sigma|}} e^{-\frac{1}{2}(x - \mu)^T \Sigma^{-1}(x - \mu)}$$

<img src="https://upload.wikimedia.org/wikipedia/commons/8/8e/MultivariateNormal.png" width="500">

Аномалиями будем считать те объекты, вероятность появления которых мала.

**Задание 3**

Допишите следующие функции:

In [ ]:
from scipy.stats import multivariate_normal
from sklearn.metrics import f1_score

# Возвращает оцененные параметры распределения
def estimateGaussian(dataset):
    #------------------------------------------------
    # Ваш код здесь
    #------------------------------------------------
    return mu, sigma

# Возвращает вероятность для каждого объекта
def multivariateGaussian(dataset, mu, sigma):
    #------------------------------------------------
    # Ваш код здесь
    #------------------------------------------------

# Возвращает лучшее значение f1 и лучший порог
def selectThresholdByCV(probs, target):
    best_epsilon = 0
    best_f1 = 0
    f = 0
    
    stepsize = (max(probs) - min(probs)) / 1000;
    epsilons = np.arange(min(probs), max(probs), stepsize)
    
    for epsilon in np.nditer(epsilons):
        #------------------------------------------------
        # Ваш код здесь
        #------------------------------------------------
    
    return best_f1, best_epsilon

Загружаем данные

In [ ]:
columns_names = ['Latency (ms)', 'Throughput (mb/s)']

# Тренировочные данные
data_train = pd.read_csv('tr_server_data.csv', names=columns_names)

# Валидационные данные
data_val = pd.read_csv('cv_server_data.csv', names=columns_names)

# Значения target на валидационных данных
data_val_tr = pd.read_csv('gt_server_data.csv', names=['target'])

In [ ]:
data_train.head()

Построим график

In [ ]:
plt.xlabel('Latency (ms)')
plt.ylabel('Throughput (mb/s)')
plt.plot(data_train.iloc[:,0], data_train.iloc[:,1],'bx')
plt.show()

**Задание 4**

1. Оцените параметры распределния по тестовым данным
2. Оцените вероятность для каждого объекта тренировочной выборки
3. Оцените вероятность для каждого объекта валидационной выборки
4. Выбирете оптимальное значение порога
5. Найдите индексы аномальных объектов в тренировочной выборке
6. Постройте график, на котороя аномальные объекты отображены крассным цветом

In [ ]:
#------------------------------------------------
# Ваш код здесь
#------------------------------------------------

## Часть 3. Одноклассовый SWM

Вариант метода опорных векторов, который отделяет выборку от начала координат. Использование ядер позволяет обойти сомнительное предположение о том, что объекты должны распологаться в дали от начала координат. По умолчанию, в качестве ядра используют лишь rbf (радиальные базисные функции), остальные ядра показывают плохие результаты.

**OneClassSVM** это скорее алгоритм поиска новизны, а не выбросов, т.к. «затачивается» под обучающую выборку.

In [ ]:
from sklearn import svm

data_train = pd.read_csv('tr_server_data.csv', names=columns_names)

**Задание 5**

1. Обучите `OneClassSVM` с параметрами:
    - `nu=0.05`
    - `kernel="rbf"`
    - `gamma=0.1`
2. Сделайте предсказание для объектов тренировочной выборки
3. Постройте график, на котороя аномальные объекты отображены крассным цветом

In [ ]:
#------------------------------------------------
# Ваш код здесь
#------------------------------------------------

## Часть 4. Изолирующий лес (Isolation Forest)

**Isolation Forest** ([оригинальная статья](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf?q=isolation-forest)) «изолирует» наблюдения следующим образом:
1. сначала, случайным образом выбирается признак,
2. затем, случайным образом выбирает *разделяющее значение* (split value) между максимальным и минимальным значениями выбранного признака.

Поскольку рекурсивное разбиение может быть представлено древовидной структурой, количество разбиений, необходимых для выделения объекта, эквивалентно длине пути от корневого узла до конечного узла.

<img src="https://miro.medium.com/max/1400/1*TqkqI2QECywBLnY6KlVvHA.png" width="600">

Эта длина пути, усредненная по лесу таких случайных деревьев, является мерой нормальности и нашей решающей функции.

$$\Large s(x, n) = 2^{-\frac{E(h(x))}{c(n)}}$$

где
- $h(x)$ - длина пути до наблюдения $x$
- $c(n)$ - средняя длина пути неудачного поиска в бинарном дереве поиска
- $n$ - количество внешних узлов

При случайном разбиение, для аномальных объектов пути получаются заметно более короткими, чем для "нормальных" объектов. Следовательно, когда лес случайных деревьев в совокупности дает более короткие длины пути для конкретных объектов, они с большой вероятностью могут быть аномалиями.

Сгенерируем данные и посмотрим на них.

In [ ]:
rng = np.random.RandomState(42)

# Тренировочные данные
X_train = 0.2 * rng.randn(1000, 2)
X_train = np.r_[X_train + 3, X_train]
X_train = pd.DataFrame(X_train, columns = ['x1', 'x2'])

# Тестовые данные - новые "нормальные" наблюдения
X_test = 0.2 * rng.randn(200, 2)
X_test = np.r_[X_test + 3, X_test]
X_test = pd.DataFrame(X_test, columns = ['x1', 'x2'])

# Аномальные данные
X_outliers = rng.uniform(low=-1, high=5, size=(50, 2))
X_outliers = pd.DataFrame(X_outliers, columns = ['x1', 'x2'])

Построим график

In [ ]:
plt.figure(figsize=(10,10))
plt.title("Data")

p1 = plt.scatter(X_train.x1, X_train.x2, c='white', s=20*4, edgecolor='k')
p2 = plt.scatter(X_test.x1, X_test.x2, c='green', s=20*4, edgecolor='k')
p3 = plt.scatter(X_outliers.x1, X_outliers.x2, c='red', s=20*4, edgecolor='k')

plt.axis('tight')
plt.xlim((-2, 5))
plt.ylim((-2, 5))
plt.legend([p1, p2, p3], ["train", "test normal", "abnormal"], loc="lower right")

plt.show()

Обучим [`IsolationForest`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html) и сделаем предсказание на наших данных

In [ ]:
from sklearn.ensemble import IsolationForest

**Задание 6**

1. Обучите `IsolationForest` со следующими параметрами:
    - `max_samples=100`
    - `contamination=0.1` - доля выбросов в наборе данных
    - `random_state=rng`
    - `behaviour="new"`
2. Посчитайте долю верных ответов (accuracy) для тестовой "нормальной" выборки и для аномалий
3. Постройте scatter plot, на котором видно какие наблюдения изолирующий лес считает аномалиями

In [ ]:
#------------------------------------------------
# Ваш код здесь
#------------------------------------------------

## Часть 5. Раздеяющие поверхности для разных алгоритмов

In [ ]:
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest

# чтобы был русский шрифт
import matplotlib
from matplotlib import rc
from scipy import stats

Сгенерируем данные

In [ ]:
n = 100 # длина выборки
n_out = 5 # число выбросов


from sklearn.datasets import make_blobs

X = make_blobs(n_samples=n, n_features=2, centers=3, random_state=2, center_box=(-8.0, 8.0))[0]
X[:n_out,:] = 20*np.random.rand(n_out, 2) - 10
y = np.ones(n)
y[:n_out] = -1

plt.scatter(X[:,0], X[:,1], c='#0000FF', label='выборка')
plt.scatter(X[:n_out,0], X[:n_out,1], c='#FF0000', label='выбросы')
plt.xlabel(u'признак 1')
plt.ylabel('признак 2')
plt.legend();

In [ ]:
xx, yy = np.meshgrid(np.linspace(-10, 10, 500), np.linspace(-10, 10, 500))

outliers_fraction = 0.1

In [ ]:
# Функция для рисования областей
def run_and_plot(clf, X, outliers_fraction, draw_legend=True, title=''):
    clf.fit(X)
    print(clf)
    
    a_prob =  clf.decision_function(X)
    threshold = stats.scoreatpercentile(a_prob, 100 * outliers_fraction)
      
    print ('ошибка  = ' + str( (clf.predict(X) != y).mean()))

    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)


    plt.figure(figsize=(8, 8))
    plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), Z.max(), 20), cmap=plt.cm.binary) # plt.cm.Blues_r cmap=plt.cm.Blues_r)
    a_ = plt.contour(xx, yy, Z, levels=[threshold], linewidths=1, colors='yellow')
    b_ = plt.scatter(X[y>0, 0], X[y>0, 1], c='white')
    c_ = plt.scatter(X[y<0, 0], X[y<0, 1], c='red')
    plt.axis('tight')
    if draw_legend:
        plt.legend(
            [a_.collections[0], b_, c_],
            [u'разделяющая поверхность', u'нормальные объекты', u'выбросы'],
            prop=matplotlib.font_manager.FontProperties(size=11), loc='upper right')
    plt.title(title)

### 5.1. OneClassSVM

In [ ]:
clf = svm.OneClassSVM(nu=0.95 * outliers_fraction + 0.05, kernel="rbf", gamma=0.1)

run_and_plot(clf, X, outliers_fraction=outliers_fraction, draw_legend=True, title='OneClassSVM, kernel="rbf", gamma=0.1')

In [ ]:
clf = svm.OneClassSVM(nu=0.95 * outliers_fraction + 0.05, kernel="poly", degree=3)
run_and_plot(clf, X, outliers_fraction=outliers_fraction, draw_legend=False, title='OneClassSVM, kernel="poly", degree=3')

### 5.2. IsolationForest

In [ ]:
clf  = IsolationForest(n_estimators=n, max_samples='auto', contamination=0.1, max_features=1.0,
                       bootstrap=False, n_jobs=1, random_state=None, verbose=0)

run_and_plot(clf, X, outliers_fraction=outliers_fraction, draw_legend=True, title='IForest, contamination=0.1, max_features=1.0')

### 5.3. EllipticEnvelope

In [ ]:
clf = EllipticEnvelope(store_precision=True, assume_centered=False, support_fraction=None, contamination=0.2, random_state=None)

run_and_plot(clf, X, outliers_fraction=outliers_fraction, draw_legend=True, title='EllipticEnvelope')

## Часть 6. Библиотека PyOD

Воспользуемся библиотекой [PyOD](https://pyod.readthedocs.io/en/latest/), в которой реализовано большое количество алгоритмов детектирования аномалий.

In [ ]:
# !pip install pyod

### 6.1. kNN

Используем метод ближайших соседий (kNN) для детектирования аномалий

In [ ]:
from pyod.utils.data import generate_data

contamination = 0.1  # процент аномалий
n_train = 200  # количество тренировочных данных
n_test = 100  # количество тестовых данных

# Сгенерируем данные
X_train, y_train, X_test, y_test = \
    generate_data(n_train=n_train,
                  n_test=n_test,
                  n_features=2,
                  contamination=contamination,
                  random_state=42)

plt.scatter(X_train[:,0], X_train[:,1], c=y_train)
plt.show()

Выброс - это точка, удаленная от соседних точек (т.е. чем дальше точка от своих соседей, тем больше вероятность того, что это выброс). 

<img src="https://miro.medium.com/max/848/1*W3anCSUzDwHzTM1BOAd5cA.png" width="400">

In [ ]:
from pyod.models.knn import KNN   # kNN detector

# обучим kNN
clf_name = 'KNN'
clf = KNN()
clf.fit(X_train)

y_train_pred = clf.labels_  # бинарные метки (0: норма, 1: выбросы)
y_train_scores = clf.decision_scores_

# предсказание на тесте
y_test_pred = clf.predict(X_test)
y_test_scores = clf.decision_function(X_test)

#### Посмотрим на качество алгоритма

In [ ]:
from pyod.utils.data import evaluate_print

print("\nOn Training Data:")
evaluate_print(clf_name, y_train, y_train_scores)

print("\nOn Test Data:")
evaluate_print(clf_name, y_test, y_test_scores)

#### Визуализируем результат

In [ ]:
from pyod.utils.example import visualize
    
visualize(clf_name, X_train, y_train, X_test, y_test, y_train_pred,
    y_test_pred, show_figure=True, save_figure=False)